# Limpieza del Dataset

La primera actividad consiste en la limpieza del dataset, de manera que éste cumpla con los requerimientos del “Tidy Data”, es decir:

1. Cada renglón o fila del dataset corresponde a una observación o individuo.
2. Cada columna del dataset corresponde a una y sólo una variable.
3. No se mezcla información de naturaleza diferente.

En el caso del dataset entregado por el INE, estos principios, sobre todo el número 1, no se respetan ya que en nuestro caso, una observación/individuo es una (y sólo una) pregunta. El resultado de esta actividad es un dataset (almacenado en un archivo CSV) que contenga los siguientes campos:

- ID del registro
- Entidad de Origen
- Edad
- Género
- Identificación con algún grupo en situación de discriminación
- Tema de la pregunta
- Texto de la pregunta

Es decir, en los casos en los que una persona envió más de una pregunta, cada pregunta debe estar en su renglón con el resto de los datos de identificación.

In [1]:
import pandas as pd
import nltk 
import re

def replace_accented(match):
    accented = 'áéíóúñüÁÉÍÓÚÑÜ'
    unaccented = 'aeiounuAEIOUNU'
    trans_table = str.maketrans(accented, unaccented)
    return match.translate(trans_table)

stop_words = nltk.corpus.stopwords.words('spanish')
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\sáéíóúñüÁÉÍÓÚÑÜ]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [replace_accented(token) for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

def repetido(s, char):
    return s.count(char) > 1

crudo_df = pd.read_excel('./Base_PreguntasFormulario-1erDebatePresidencial2024.xlsx')

crudo_df['Edad:'] = crudo_df['Edad:'].str.replace(' años', '').astype(int)

crudo_df.rename(columns = {
        '¿Cuál es tu entidad de origen?': 'entidad', 
        'Edad:': 'edad',
        'Género:': 'genero',
        '¿Te identificas con alguno de los siguientes Grupos en Situación de Discriminación?': 'grupo_discriminacion',
        'Indica el tema de tu pregunta:': 'tema',
        'Escribe tu pregunta:': 'pregunta',
        '¿Deseas agregar otra pregunta?': 'extra',
        'Indica el tema de tu pregunta: (2)': 'tema2',
        'Escribe tu pregunta: (2)': 'pregunta2',
        '¿Deseas agregar otra pregunta? (2)': 'extra2',
        'Indica el tema de tu pregunta: (3)': 'tema3',
        'Escribe tu pregunta: (3)': 'pregunta3',
        '¿Deseas agregar otra pregunta? (3)': 'extra3',
        'Indica el tema de tu pregunta: (4)': 'tema4',
        'Escribe tu pregunta: (4)': 'pregunta4',
        '¿Deseas agregar otra pregunta? (4)': 'extra4',
        'Indica el tema de tu pregunta: (5)': 'tema5',
        'Escribe tu pregunta: (5)': 'pregunta5',
        '¿Deseas agregar otra pregunta? (5)': 'extra5',
        'Indica el tema de tu pregunta: (6)': 'tema6',
        'Escribe tu pregunta: (6)': 'pregunta6',
        'Fecha de entrada': 'fecha'
    }, inplace = True)

todo_preguntas = []
todo_preguntas.append(crudo_df[['entidad', 'edad', 'genero', 'grupo_discriminacion', 'tema','pregunta', 'fecha']])

#Asignación de los Archivos a la Lista
for idx in range(2, 6+1):
    todo_preguntas.append(crudo_df[['entidad', 'edad', 'genero', 'grupo_discriminacion', 'tema'+str(idx),'pregunta'+str(idx), 'fecha']].rename(columns={'tema'+str(idx): 'tema', 'pregunta'+str(idx): 'pregunta'}))
    
#Concatenación de los Elementos de la Lista a un DataFrame
limpio_df = pd.concat(todo_preguntas)
limpio_df = limpio_df.dropna()
limpio_df.drop_duplicates(inplace=True)
 
limpio_df['grupo_discriminacion'] = limpio_df['grupo_discriminacion'].str.replace('Selecciona','N/A')
limpio_df['fecha'] = limpio_df['fecha'].replace({'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04', 'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08', 'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'}, regex=True)
limpio_df['fecha'] = pd.to_datetime(limpio_df['fecha'], format='%d %m, %Y %I:%M %p')

limpio_df['pregunta_norm'] = list(map(normalize_document, limpio_df['pregunta']))
limpio_df['pregunta_norm'] = limpio_df['pregunta_norm'].astype(str)

limpio_df.info()
display(limpio_df)

limpio_df.to_csv('./Base_limpia.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 22890 entries, 0 to 13456
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   entidad               22890 non-null  object        
 1   edad                  22890 non-null  int32         
 2   genero                22890 non-null  object        
 3   grupo_discriminacion  22890 non-null  object        
 4   tema                  22890 non-null  object        
 5   pregunta              22890 non-null  object        
 6   fecha                 22890 non-null  datetime64[ns]
 7   pregunta_norm         22890 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(6)
memory usage: 1.5+ MB


,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha,pregunta_norm
0,Ciudad de México,43,Femenino,N/A,No discriminación y grupos vulnerables,"Candidat@, cómo apoyaría a los productores de ...",2024-03-22 00:04:00,candidat como apoyaria productores nopal alcal...
1,Aguascalientes,14,Masculino,N/A,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00,mexico muchas personas quejan sistema salud co...
2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00,estrategias planes concretos proponen asegurar...
3,México,34,Femenino,N/A,Salud,Dentro de su plan de gobierno ¿qué acciones es...,2024-03-22 00:01:00,dentro plan gobierno acciones dirigidas crear ...
4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00,sigue corrupcion pesar saber donde si origuina
...,...,...,...,...,...,...,...,...
13314,Morelos,69,Masculino,Personas adultas mayores,Transparencia,"La administracion y control, de puertos, aerop...",2024-02-20 19:47:00,administracion control puertos aeropuertos via...
13413,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00,hara poder favorecer mexicanos vulnerables alg...
13432,Guanajuato,25,Masculino,N/A,Violencia en contra de las mujeres,Se le cerraran las puertas a las mujeres en pa...,2024-02-20 11:59:00,cerraran puertas mujeres palacio gobierno
13438,Puebla,24,Masculino,N/A,No discriminación y grupos vulnerables,Qué apoyo se les brindará a los grupos más mar...,2024-02-20 11:41:00,apoyo brindara grupos marginados pais
